# Syncing Langfuse Score

> 문서 작성일: 2025.03.31

GIP에서 제공하는 평가 기능을 사용할때, evaluator 생성시 지정한 `metric` 설정이 있다면, experiment run 및 auto evaluation config 실행 결과로 나오는 score 및 rational 정보를 Langfuse의 [score](https://langfuse.com/docs/scores/custom)로 동기화 할 수 있습니다.

## 필수 조건

### Experiment > `type` 구성

experiment의 `type` 값이 다음 중 하나에 해당되어야 합니다.

1. `langfuse_dataset`
2. `langfuse_observation`
3. `langfuse_trace`

또한, `langfuse_dataset`의 경우 dataset item 내에 `source_observation_id` 혹은 `source_trace_id` 정보가 포함되어 있어야 하며, 이에 대한 정보가 없으면 score 동기화는 진행되지 않습니다.


### Evaluator > `metric` 구성

experiment run 및 auto evaluation config 생성시에 지정한 evaluator에는 다음과 같이 metric 정보가 구성되어 있어야 합니다.

1. Category metric
    ```json
   {
      "id": "f6e83127-d730-4d33-aec1-602af05c804c",
      "workspace_id": "cb0ed07c-75cd-43f6-8c76-ff994eec4ae3",
      "name": "[set_1] LLM Judge + category metric evaluator",
      "metric": {
        "type": "category",
        "config": [
          {
            "index": 0,
            "name": "incorrect"
          },
          {
            "index": 1,
            "name": "correct"
          }
        ]
      },
      "type": "llm_judge",
      "evaluation_config": {
        "judge_model_seq_id": 19,
        "judge_model_public_id": "openai/gpt-4",
        "judge_model_parameters": {
          "top_p": 0.5,
          "temperature": 0.5,
          "max_tokens": 150
        },
        "judge_prompt": "You are an impartial AI judge ..."
      }
    }

    ```
2. Numeric metric
    ```json
   {
      "id": "00f68e92-e98f-4635-aecd-40be24f2bb10",
      "workspace_id": "cb0ed07c-75cd-43f6-8c76-ff994eec4ae3",
      "name": "[set_2] LLM Judge + numeric metric evaluator",
      "metric": {
        "type": "numeric",
        "config": {
          "min": {
            "value": 0.0
          },
          "max": {
            "value": 1.0
          }
        }
      },
      "type": "llm_judge",
      "evaluation_config": {
        "judge_model_seq_id": 19,
        "judge_model_public_id": "openai/gpt-4",
        "judge_model_parameters": {
          "top_p": 0.5,
          "temperature": 0.5,
          "max_tokens": 150
        },
        "judge_prompt": "You are an impartial AI judge. ...."
      }
    }

    ```
3. Boolean metric
    ```json
   {
      "id": "cc15637a-1138-4263-a4ea-3b785d7b00fe",
      "workspace_id": "cb0ed07c-75cd-43f6-8c76-ff994eec4ae3",
      "name": "[set_3] LLM Judge + bool metric evaluator",
      "metric": {
        "type": "bool"
      },
      "type": "llm_judge",
      "evaluation_config": {
        "judge_model_seq_id": 19,
        "judge_model_public_id": "openai/gpt-4",
        "judge_model_parameters": {
          "top_p": 0.5,
          "temperature": 0.5,
          "max_tokens": 150
        },
        "judge_prompt": "You are an impartial AI judge ..."
      }
    }
    ```

## 사전 준비

우선 다음의 python 패키지가 설치되어 있어야 합니다.

- `requests`
- `python-dotenv`

환경 변수 파일(`.env`)를 사용하기 위해서는 `.env` 파일에 다음과 같은 내용을 채워주세요.

```
GIP_CONSOLE_HOST=https://dev-console-api.platform.a15t.com
GIP_CONSOLE_ENTERPRISE_API_KEY=
GIP_WORKSPACE_ID=
GIP_EXPERIMENT_ID=
GIP_EVALUATOR_ID=
GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID=
```

> 만약 `GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID` 값이 없으면 문서 내의 `Langfuse credential 연동` 부분을 참고해서 설정하시면 됩니다.

In [ ]:
import os

from dotenv import load_dotenv


load_dotenv() # Optional: 환경 변수 파일(.env)을 불러옵니다


GIP_CONSOLE_HOST = os.getenv('GIP_CONSOLE_HOST', 'https://dev-console-api.platform.a15t.com')
GIP_CONSOLE_ENTERPRISE_API_KEY = os.getenv('GIP_CONSOLE_ENTERPRISE_API_KEY')
GIP_WORKSPACE_ID = os.getenv('GIP_WORKSPACE_ID')

GIP_EXPERIMENT_ID = os.getenv('GIP_EXPERIMENT_ID')
GIP_EVALUATOR_ID = os.getenv('GIP_EVALUATOR_ID')

GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID = os.getenv('GIP_LANGFUSE_INTEGRATED_SERVICE_CREDENTIAL_ID')

### 인증 방법 관련 안내

GIP Console의 API를 호출하는 데에는 두가지 방법이 있습니다.

1. Enterprise API Key 사용
2. GIP Console JWT Token 사용

이 문서는 enterprise API key를 사용하는 것을 기준으로 evaluation API를 활용하는 방법에 대해서 안내하고 있으며, 필요하신 경우 인증 헤더 정보를 GIP Console JWT Token을 사용하도록 변경하시면 됩니다.

#### 1. Enterprise API Key 사용

In [ ]:
import requests


response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
)

response.raise_for_status()

#### 2. GIP Console JWT Token 사용

In [ ]:
import requests


# GIP Console 로그인을 통해서 JWT Token을 발급받는 과정
response = requests.post(
    url='https://cognito-idp.ap-northeast-2.amazonaws.com',
    headers={
        'Content-Type': 'application/x-amz-json-1.1',
        'X-Amz-Target': 'AWSCognitoIdentityProviderService.InitiateAuth',
    },
    json={
        'AuthParameters': {
            # GIP Console 로그인 시에 사용하는 계정 정보 기재
            'USERNAME': 'keep-secret',
            'PASSWORD': 'keep-secret',
        },
        'AuthFlow': 'USER_PASSWORD_AUTH',
        'ClientId': '4dm722bemutkce2500cck1bbk4', # dev 환경용 client ID -> prod 환경의 GIP Console API 호출시에는 별도 문의 필요
    },
)

response.raise_for_status()

auth_response = response.json()
id_token = auth_response['AuthenticationResult']['IdToken']

# GIP Console 로그인을 통해서 얻은 JWT Token을 사용해서 GIP Console API 호출
response = requests.get(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/models',
    headers={'Authorization': f'Bearer {id_token}'}
)

response.raise_for_status()

## 요청 예시

### Experiment Run

In [ ]:
import requests


response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{GIP_EXPERIMENT_ID}/runs',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': GIP_EVALUATOR_ID,
        'display_name': 'Langfuse Score Syncing Example 1',
        'target_data_converter': None,
        'metadata': {
            'langfuse_integration_config': {
                'score_name': 'sync_score_test_v1', # NOTE: 사용하고 싶은 Langfuse score 이름 기재
                'sync_score': True,
            }
        },
    },
)

response.raise_for_status()

### Auto Evaluation Config

In [ ]:
response = requests.post(
    url=f'{GIP_CONSOLE_HOST}/api/workspaces/{GIP_WORKSPACE_ID}/evals/experiments/{GIP_EXPERIMENT_ID}/auto-evaluations',
    headers={'X-Gipc-Api-Key': GIP_CONSOLE_ENTERPRISE_API_KEY},
    json={
        'evaluator_id': GIP_EVALUATOR_ID,
        'enabled': True,
        'name': 'Langfuse Score Syncing Example 2 (auto)',
        'schedule': '0 23 * * *', # cron expression: 매일 23시에 평가를 진행
        'timezone': 'Asia/Seoul',  # default: `Etc/UTC`
        'target_data_converter': None,
        'metadata': {
            'langfuse_integration_config': {
                'score_name': 'sync_score_test_v2', # NOTE: 사용하고 싶은 Langfuse score 이름 기재
                'sync_score': True,
            }
        },
    },
)

response.raise_for_status()